In [1]:
import torch

print(torch.cuda.is_available())

for i in range(0, torch.cuda.device_count()):
    print(f'device[{i}]: {torch.cuda.get_device_name(i)}')

True
device[0]: NVIDIA GeForce RTX 4070 Ti


In [6]:
import fiftyone as fo
import fiftyone.zoo as foz

print(foz.list_zoo_datasets())

dataset = foz.load_zoo_dataset('sama-coco', split='validation', dataset_name="sama-coco-valid-makrela")

['activitynet-100', 'activitynet-200', 'bdd100k', 'caltech101', 'caltech256', 'cifar10', 'cifar100', 'cityscapes', 'coco-2014', 'coco-2017', 'fashion-mnist', 'fiw', 'hmdb51', 'imagenet-2012', 'imagenet-sample', 'kinetics-400', 'kinetics-600', 'kinetics-700', 'kinetics-700-2020', 'kitti', 'kitti-multiview', 'lfw', 'mnist', 'open-images-v6', 'open-images-v7', 'quickstart', 'quickstart-geo', 'quickstart-groups', 'quickstart-video', 'sama-coco', 'ucf101', 'voc-2007', 'voc-2012']
Found annotations at '/home/jovyan/fiftyone/sama-coco/raw/sama_coco_validation.json'
Images already downloaded
Existing download of split 'validation' is sufficient
Loading existing dataset 'sama-coco-valid-makrela'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use


In [7]:
dataset.persistent = True

In [8]:
config = fo.AppConfig()
config.color_by = 'value'

session = fo.launch_app(dataset, auto=False, config=config)

print('Head to http://localhost:5151/')

Session launched. Run `session.show()` to open the App in a cell output.
Head to http://localhost:5151/


In [9]:
%pip install distinctipy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [11]:
import distinctipy

def clamp(x):
  x = int(x * 255)
  return max(0, min(x, 255))

def generate_color_scheme(dataset: fo.Dataset):
    valueColors: list[dict[str,str]] = []

    colors =  distinctipy.get_colors(len(dataset.default_classes))

    for index, item in enumerate(dataset.default_classes):
        selected_color = colors[index]
        color = "#{0:02x}{1:02x}{2:02x}".format(clamp(selected_color[0]), clamp(selected_color[1]), clamp(selected_color[2]))
        itemColors = { "value": str(item), "color": color }

        valueColors.append(itemColors)

    color_scheme = fo.ColorScheme(
        fields=[
            {
                "path": "ground_truth",
                "colorByAttribute": "label",
                "valueColors": valueColors
            }
        ]
    )

    return color_scheme

# session.color_scheme = color_scheme

In [ ]:
!fiftyone plugins download \
    https://github.com/voxel51/fiftyone-plugins \
    --plugin-names @voxel51/io

In [13]:
test_dataset = fo.Dataset.from_dir(labels_path='annotations.xml', dataset_dir='./dataset/', data_path='images', dataset_type=fo.types.CVATImageDataset)
test_dataset.persistent = True

 100% |███████████████████| 12/12 [46.6ms elapsed, 0s remaining, 257.6 samples/s]     


In [14]:
session.color_scheme = generate_color_scheme(test_dataset)

In [15]:
import fiftyone.brain as fob

fob.compute_uniqueness(test_dataset)

 100% |████|  100.6Mb/100.6Mb [565.0ms elapsed, 0s remaining, 178.0Mb/s]      
Computing embeddings...
 100% |███████████████████| 12/12 [6.1s elapsed, 0s remaining, 2.0 samples/s] 
Computing uniqueness...
Uniqueness computation complete
